# Hydraulic Model

In [ ]:
import numpy as np

import json

import matplotlib.pyplot as plt

# import function from another jupyter notebook
import sys
sys.path.insert(1, '1_Functions')

import Function_complex

### Inputs
For varying soil matric potentials hb and transpiration rates E we calculate the leaf water potential hleaf

In [2]:
dx, dy, dz = 100, 100, 0.3

h_b = -1*np.arange(10,15010+100,dx, dtype=int) #bulk soil water potential in cm heads
E = 3e-6*np.arange(0,3000+1,dz) #transpiration rate in cm3 s**-1

root-plant properties

In [3]:
R_root = 0.12e7 # hPa cm**-3 s; for the blue 0.08; for the green 0.12; inverse of conductance
r0 = 0.05 # cm root radius in cm
#r2 = 1. # bulk+root+rhizo soil radius in cm
V_root = np.pi*3000
L = 30000. # root length in cm - for the blue 1000 - for the green 10000
r2 = np.sqrt(V_root/(np.pi*L))

Brooks-Corey parameters for the soil hydraulic properties

In [4]:
h0 = -10. # this is alfa [cm**-1]
l = 1.5
#ths = 0.45 # not needed in the code, but useful if the decreasing water content needs to be calculated
#thr = 0.01 # not needed in the code, but useful if the decreasing water content needs to be calculated
k0 = 3e-3 # cm/s - for the blue 0.2 (clay); hydraulic conductivity for soil = k_sat?
tau = 2.5 # corresponding to 2+l*(a+2) with l exp for theta(h) and a tortousity

Soil hydraulic properties

In [5]:
xx=np.array([   [0.09, 0.475, 0.0268, 37.31, 0.131, 1.44, (0+45)/2, (45-0)/2], #clay
                [0.075, 0.366, 0.0386, 25.91, 0.194, 5.52, (20+45)/2, (45-20)/2], #clay loam
                [0.027, 0.463, 0.0897, 11.15, 0.22, 31.68, (23+52)/2, (52-23)/2], #loam
                [0.035,	0.437,	0.115,	8.70,	0.474,	146.64, (70+86)/2, (86-70)/2], #loamy sand
                [0.02, 0.437, 0.138, 7.25, 0.592, 504, (86+100)/2, (100-86)/2], #sand
                [0.068, 0.398, 0.0356, 28.09, 0.25, 10.32, (45+80)/2, (80-45)/2], #sandy clay loam
                [0.041,	0.453,	0.0682,	14.66,	0.322,	62.16,   (50+70)/2, (70-50)/2],  #sandy loam
                [0.015, 0.501, 0.0482, 20.75, 0.211, 16.32, (20+50)/2, (50-20)/2],  #silt loam
                [0.04, 0.471, 0.0307, 32.57, 0.151, 3.6, (0+20)/2, (20-0)/2] #silty clay loam
    ])

nb_soils = len(xx)

soil_types = ["Clay", "Clay Loam", "Loam", "Loamy Sand", "Sand", "Sandy Clay Loam", "Sandy Loam", "Silt Loam", "Silty Clay Loam"]

In [6]:
thr_xx = xx[:,0]
ths_xx = xx[:,1]
h0_xx  = -1*xx[:,3]
l_xx   = xx[:,4]
k0_xx  = xx[:,5] / (24*60*60) # cm/s - for the blue 0.2; 3*10^-3
tau_xx = 2+3*l_xx #corresponding to 2+l*(a+2) with l exp for theta(h) and a tortuosity

hxx = np.array([thr_xx, ths_xx, h0_xx, l_xx, k0_xx, tau_xx])

In [7]:
soil_Data = {}

soil_Data = {
    soil: {
        "thr": thr_xx[i],
        "ths": ths_xx[i],
        "h0": h0_xx[i],
        "k0": k0_xx[i],
        "tau": tau_xx[i]
    } 
    for i, soil in enumerate(soil_types)
}

#soil_Data

In [8]:
# add test soil
soil_Data["Test Soil"] = {"thr":np.array(0.01), "ths":np.array(0.45), "h0":np.array(-10.), "k0":np.array(3e-3), "tau":np.array(2.5)}

radial rhizosphere domain

In [9]:
r = np.arange(r0,r2+0.01,0.01).T
dr = float((r2-r0)/(len(r)-1)) # separation steps

parameters for cavitation; if you plot the harmonic mean of 1/Rroot and k_x you get a conductance that maximum is the one of root and if hx decreases the conductivity drops 

In [10]:
k0_x = 1/R_root # hPa-1 cm^3 s-1; conductance of roots (assumed to be uniform)
h0_x = -25000 # in cm (or 10^-4 Mpa) approximately equal p50; water head of xylem
tau_x = 5
#k_x = k0_x*(h_leaf/h0_x)^-tau_x # this would be the xylem conductance; BC model for xylem

In [11]:
# Store all plant parameters in a list
params_plant = R_root, h0_x, tau_x, L

### Soil Analysis
Check dependecies between different soil properties

In [ ]:
plt.scatter(tau_xx, h0_xx)
plt.xlabel('tau')
plt.ylabel('h0')

In [ ]:
plt.scatter(tau_xx, k0_xx)
plt.xlabel('tau')
plt.ylabel('k0')

In [ ]:
plt.scatter(h0_xx, k0_xx)
plt.xlabel('h0')
plt.ylabel('k0')

### Caclucate SOL and iso lines for each soil
Using the full SPAC function from Function_complex

In [12]:
iso_soil = np.array([-10, -3000, -5000, -7000, -10000, -15000]) # values for iso-hsoil lines
iso_leaf = np.array([-2000, -5000, -7500, -10000, -15000, -30000]) # values for iso-hleaf lines
iso_E = np.array([0.1, 0.2, 0.4, 0.6, 0.8]) * max(E) # values for iso-E lines

for soil in soil_Data:
    print(soil)

    tau = float(soil_Data[soil]["tau"])
    h0 = float(soil_Data[soil]["h0"])
    k0 = float(soil_Data[soil]["k0"])

    params_soil = r0, h0, k0, tau
    
    SOL = Function_complex.Calc_SOL(E, h_b, params_soil, params_plant)

    soil_Data[soil]["traject"] = SOL

    for _, hsoil_val in enumerate(iso_soil):
        soil_Data[soil][f"trajecthsoil_{hsoil_val}"] = Function_complex.Calc_hleaf(E, hsoil_val, params_soil, params_plant)

    for _, hleaf_val in enumerate(iso_leaf):

        iso_leaf_h = []
        iso_leaf_E = []
    
        for i, hb in enumerate(h_b):
            h_leaf = Function_complex.Calc_hleaf(E, hb, params_soil, params_plant) # Calculate the iso psi soil curve

            idx_leaf = np.nanargmin(np.abs(h_leaf - hleaf_val))

            if E[idx_leaf] != 0:
                iso_leaf_h.append(hb)
                iso_leaf_E.append(E[idx_leaf].min())

        soil_Data[soil][f"trajecthleaf_{hleaf_val}"] = np.column_stack([np.array(iso_leaf_h), np.array(iso_leaf_E)])

    for _, E_val in enumerate(iso_E):

        idx_E = np.nanargmin(np.abs(E - E_val))

        iso_E_leaf = []

        for i, hb in enumerate(h_b):
            h_leaf = Function_complex.Calc_hleaf(E, hb, params_soil, params_plant) # Calculate the iso psi soil curve

            iso_E_leaf.append(h_leaf[idx_E])

        soil_Data[soil][f"trajectE_{E_val/np.max(E)}"] = np.array(iso_E_leaf)

Clay
Clay Loam
Loam


/Users/alien/opt/miniconda3/envs/soil/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1245: RuntimeWarning: divide by zero encountered in divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/Users/alien/opt/miniconda3/envs/soil/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1245: RuntimeWarning: invalid value encountered in divide
  a = -(dx2)/(dx1 * (dx1 + dx2))
/Users/alien/opt/miniconda3/envs/soil/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1246: RuntimeWarning: divide by zero encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/Users/alien/opt/miniconda3/envs/soil/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1246: RuntimeWarning: invalid value encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/Users/alien/opt/miniconda3/envs/soil/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1247: RuntimeWarning: divide by zero encountered in divide
  c = dx1 / (dx2 * (dx1 + dx2))
/Users/alien/opt/miniconda3/envs/soil/lib/pyt

Loamy Sand
Sand
Sandy Clay Loam
Sandy Loam
Silt Loam
Silty Clay Loam
Test Soil


Store data outside python file to avoid computation everytime

In [ ]:
soil_Data_json = {soil: {param: values.tolist() for param, values in params.items() if not param == "c_soil"} 
                  for soil, params in soil_Data.items()}

with open("../../2_Data/1_SoilData/soil_Data.json", "w") as f:
    json.dump(soil_Data_json, f, indent=4)

Load data from json file

In [ ]:
""" with open('soil_Data.json') as f:
    soil_Data = json.load(f) """

### Outputs

In [ ]:
# Plot from the leaf perspective

plt.figure(1)
plt.subplots_adjust(hspace=5)
fig, axs = plt.subplots(figsize=(12, 12))

for i, soil in enumerate(soil_Data):
    trajectory2 = soil_Data[soil]["traject"]

    trajecthsoil1 = soil_Data[soil]["trajecthsoil_-10"]
    trajecthsoil30 = soil_Data[soil]["trajecthsoil_-3000"]
    trajecthsoil50 = soil_Data[soil]["trajecthsoil_-5000"]
    trajecthsoil70 = soil_Data[soil]["trajecthsoil_-7000"]
    trajecthsoil100 = soil_Data[soil]["trajecthsoil_-10000"]
    trajecthsoil150 = soil_Data[soil]["trajecthsoil_-15000"]

    ax = plt.subplot(4, 3, i + 1)

    ax.set_title(soil) # title with soil name
    ax.plot(-trajecthsoil1 * 1e-4, E / np.nanmax(E), 'k', linewidth=2)
    ax.plot(-trajecthsoil30 * 1e-4, E / np.nanmax(E), 'k--', linewidth=1)
    ax.plot(-trajecthsoil50 * 1e-4, E / np.nanmax(E), 'k--', linewidth=1)
    ax.plot(-trajecthsoil70 * 1e-4, E / np.nanmax(E), 'k--', linewidth=1)
    ax.plot(-trajecthsoil100 * 1e-4, E / np.nanmax(E), 'k--', linewidth=1)
    ax.plot(-trajecthsoil150 * 1e-4, E / np.nanmax(E), 'k--', linewidth=1)
    ax.plot(-trajectory2[:, 1] * 1e-4, trajectory2[:, 2] / np.nanmax(E), 'r', linewidth=2)
    #ax.ylabel(r'$E \ [cm^3 s^{-1}]$', fontsize=18)
    ax.set_xlabel(r'$-\psi_{leaf} \ [MPa]$')
    ax.set_ylabel(r'$E \ [-]$')
    #ax.ylabel(r'$E \ [-]$', fontsize=18)
    #ax.xlabel(r'$-\psi_{leaf} \ [MPa]$', fontsize=18)

    ax.set_xbound(lower=-0.1, upper=3)

plt.tight_layout()
plt.show()

In [ ]:
# Plot from the soil perspective

plt.figure(2)
plt.subplots_adjust(hspace=5)
fig, axs = plt.subplots(figsize=(12, 12))

for i, soil in enumerate(soil_Data):
    trajectory2 = soil_Data[soil]["traject"]

    trajecthleaf2000 = soil_Data[soil]["trajecthleaf_-2000"]
    trajecthleaf5000 = soil_Data[soil]["trajecthleaf_-5000"]
    trajecthleaf7500 = soil_Data[soil]["trajecthleaf_-7500"]
    trajecthleaf10000 = soil_Data[soil]["trajecthleaf_-10000"]
    trajecthleaf15000 = soil_Data[soil]["trajecthleaf_-15000"]
    trajecthleaf30000 = soil_Data[soil]["trajecthleaf_-30000"]

    ax = plt.subplot(4, 3, i + 1)

    ax.set_title(soil) # title with soil name
    ax.plot(-trajecthleaf7500[:,0] * 1e-4, trajecthleaf7500[:,1], 'k')
    ax.plot(-trajecthleaf2000[:,0] * 1e-4, trajecthleaf2000[:,1], 'k')
    ax.plot(-trajecthleaf5000[:,0] * 1e-4, trajecthleaf5000[:,1], 'k')
    ax.plot(-trajecthleaf10000[:,0] * 1e-4, trajecthleaf10000[:,1], 'k')
    ax.plot(-trajecthleaf15000[:,0] * 1e-4, trajecthleaf15000[:,1], 'k')
    ax.plot(-trajecthleaf30000[:,0] * 1e-4, trajecthleaf30000[:,1], 'k')
    ax.plot(-trajectory2[:, 0] * 1e-4, trajectory2[:, 2], 'r', linewidth=2)
    #ax.ylabel(r'$E \ [cm^3 s^{-1}]$', fontsize=18)
    ax.set_xlabel(r'$-\psi_{soil} \ [MPa]$')
    ax.set_ylabel(r'$E \ [-]$')
    #ax.ylabel(r'$E \ [-]$', fontsize=18)
    #ax.xlabel(r'$-\psi_{leaf} \ [MPa]$', fontsize=18)

plt.tight_layout()
plt.show()